<a href="https://colab.research.google.com/github/CherpanovNazim/learn-llm/blob/main/notebooks/00_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* [GitHub Repo](https://github.com/CherpanovNazim/learn-llm)

In [ ]:
#clone git repository
!git clone -q https://github.com/CherpanovNazim/learn-llm.git

In [ ]:
# wait ~3 min for installations
%%time

!pip install -qU openai==1.40.3 vllm==0.5.4 transformers==4.44.0

In [ ]:
# wait ~2 min for installations
%%time

import json
import sys

# Load the default model
DEFAULT_MODEL = json.load(open('learn-llm/configs/llama_3_8B_instruct_awq.json', 'r'))

#run VLLM
!nohup vllm serve {DEFAULT_MODEL['model']} --quantization awq --max-model-len=4096 > vllm.log &
!tail -f vllm.log | grep -q "Uvicorn running" && echo "Now you can start using the model"

# Why LLAMA 3 ?

* We mainly use GPT 3.5 or GPT 4 models for production use-cases
* For this course we use LLAMA 3 8B model for educational purposes
* LLAMA 3 is smaller, but still very powerful
* We use OpenAI-compatible API for our LLAMA 3 - so this should would work for GPT 3.5 or GPT 4 as well

In [ ]:
!python3 learn-llm\notebooks\utils\explainer.py
sys.path.append('learn-llm/notebooks/utils')

from explainer import Explainer

explain = Explainer(DEFAULT_MODEL)
# use this class if you want to get some explanations

explain("difference between LLMs and other deep learning NLP models in a small table")

In [ ]:
import openai
import os

# Set the base URL and API key
# For production apps it's preferable to use some secret management system and don't store the key in git repo :)

client = openai.OpenAI(
    base_url = DEFAULT_MODEL['api_base'],
    api_key = DEFAULT_MODEL['api_key']
    )

## Completion VS ChatCompletion API:  
There is also 2 variants of API or how to use these models, for example in Python: `Completion` and `ChatCompletion`.  

## [Completion API](https://platform.openai.com/docs/guides/gpt/completions-api):  
For the most common case you need to specify:
* model name
* prompt that will have the task for the model and should be a string


In [ ]:
response = client.completions.create(
    model=DEFAULT_MODEL["model"],
    prompt="Write a tagline for an ice cream shop.",
    temperature=0,
    max_tokens=128,
    n=1)

In [ ]:
# Example of an output as an completion object:
print(response)

In [ ]:
print(response.choices[0].text.strip())

Let's cover most important parameters for API:  
- `temperature` -- aka randomness or 'creativity' of an output, `[0, 1]`. 0 -- no random, the output will be the same all the executions. 1 -- the output will be very different each execution.  

In [ ]:
response = client.completions.create(
    model=DEFAULT_MODEL["model"],
    prompt="Write a tagline for an ice cream shop.",
    temperature=0.75,
    max_tokens=128,
    n=1,
)
print(response.choices[0].text.strip())

* `max_tokens` -- limits number of tokens in the output. It works like too low values will shrink your output, but not making the model to generate very short output!

In [ ]:
response = client.completions.create(
    model=DEFAULT_MODEL["model"],
    prompt="Write a tagline for an ice cream shop.",
    temperature=0.75,
    max_tokens=5,
    n=1,
)
print(response.choices[0].text.strip())

* `n` -- how many output variants to generate

In [ ]:
response = client.completions.create(
    model=DEFAULT_MODEL["model"],
    prompt="Write a tagline for an ice cream shop.",
    temperature=0.75,
    max_tokens=128,
    n=2,
)

In [ ]:
print(response.choices[0].text.strip())

In [ ]:
print(response.choices[1].text.strip())

Be aware, it's meaningless to use `n > 1` and `temperature == 0` since you will get `n` equal results.

Other parameters you can check on official [OpenAI page](https://platform.openai.com/docs/api-reference/chat/create#chat/create-max_tokens:~:text=given%20chat%20conversation.-,Request%20body,-model).

## [ChatCompletion API](https://platform.openai.com/docs/guides/gpt/chat-completions-api):  

For the most common case you need to specify:
* model name
* messages in the form of list of dictionaries; they will emulate a chat between a user and an assistant

In [ ]:
response = client.chat.completions.create(
  model=DEFAULT_MODEL["model"],
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"},
    ]
)

In [ ]:
# Example of an output completion object:
print(response)

In [ ]:
print(response.choices[0].message.content.strip())

The **role** here is specifying who exactly sends the message, whether its from Human or from AI.  
- **system role** is not required, but helpful to specify how a model should consider itself.  
[From OpenAI documentation](https://platform.openai.com/docs/guides/gpt/chat-completions-api#:~:text=The%20system%20message,a%20helpful%20assistant.%22):  
"The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."  
- **human role** are possible inputs from the user helping a model to 'fit' on them
- **assistant** -- examples of a desired behavior on the user's inputs

# 📝 Prompt Engineering:  
In this section we will cover:
* Existing best techniques for prompt engineerging
* Completion prompts for some Data Science tasks  


*[References] This section is based on [DeepLearning.ai course](https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/1/introduction?_gl=1*tjnub9*_ga*MzgyMDU1MDc3LjE2ODMwNTU2MzE.*_ga_PZF1GBS1R1*MTY4Nzk0NjM3My45LjEuMTY4Nzk0Nzk4Ni4zOC4wLjA.).*

In [ ]:
def generate_response(prompt: str, system_prompt: str = None):
    if system_prompt is None:
        system_prompt = "Just follow user instructions and don't communicate like \"Sure!\" or \"I hope this helps\""

    response = client.chat.completions.create(
    model=DEFAULT_MODEL["model"],
    temperature=0,
    max_tokens=256,
    messages=[{"role": "system", "content": system_prompt},
              {"role": "user", "content": prompt}],
    )

    return response.choices[0].message.content.strip()

## Best practices:  
Now let's observe and try different techniques helping you to construct a comprehensive and effective prompt.

**1. Use delimiters to indicate distinct parts of the input clearly:**

Delimiters can be anything like: ```, """, < >, \<tag> \</tag>. It could help model focusing on specific parts of the text:

In [ ]:
text = """
Jupiter is the fifth planet from the Sun and the largest in the Solar System.
It is a gas giant with a mass one-thousandth that of the Sun, but two-and-a-half
times that of all the other planets in the Solar System combined.
Jupiter is one of the brightest objects visible to the naked eye in the night sky,
and has been known to ancient civilizations since before recorded history.
It is named after the Roman god Jupiter.[19] When viewed from Earth, Jupiter can be
bright enough for its reflected light to cast visible shadows,[20] and is on average
the third-brightest natural object in the night sky after the Moon and Venus.
"""

prompt = f"""Summarize the text delimited by triple backticks into a single sentence. ```{text}``` """
print(generate_response(prompt=prompt))

**2. Ask for a structured output:**

You can as about JSON, HTML or any other reasonable format. JSON could be useful in many cases, let's try it:

In [ ]:
prompt = f"""
Generate a list of three made-up book titles along
with their authors and genres. Provide them in JSON
format with the following keys: book_id, title, author, genre.
"""

print(generate_response(prompt=prompt))

**3. Ask the model to check whether conditions are satisfied:**  
It's like `IF-ELSE` statement inside a prompt:

In [ ]:
recipe_w_steps = f"""
Making a cup of tea is easy! First, you need to get some
 water boiling. While that's happening,
grab a cup and put a tea bag in it. Once the water is
hot enough, just pour it over the tea bag.
Let it sit for a bit so the tea can steep. After a
few minutes, take out the tea bag. If you
like, you can add some sugar or milk to taste.
And that's it! You've got yourself a delicious
cup of tea to enjoy.
"""

recipe_wo_steps = f"""
Making a cup of tea is easy! Just do it!
"""


prompt = """
You will be provided with text delimited by triple quotes. If it contains some instructions, re-write those instructions in the following format:
Step 1 - ... Step 2 - ... Step N - ...

If provided text does not contain any instructions, simply write \"No steps provided.\".

Text: \"\"\"{recipe}\"\"\"
"""

print("Steps provided:\n", generate_response(prompt=prompt.format(recipe=recipe_w_steps)))
print("\n", "-"*30, "\n")
print("Steps missed:\n", generate_response(prompt=prompt.format(recipe=recipe_wo_steps)))

**4. Focus on specific aspects:**  
Specifically ask a model to pay more attention on some details according to the task:

In [ ]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture,
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100)
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black,
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities:
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

prompt = f"""
Your task is to help a marketing team create a description for a retail website of a product based on a technical fact sheet.
Write a product description based on the information provided in the technical specifications delimited by triple backticks.

The description is intended for furniture retailers, so should be technical in nature and focus on the materials the product is constructed from.

Use at most 50 words.
Technical specifications: ```{fact_sheet_chair}```
"""

print(generate_response(prompt=prompt))

**5. Specify the model’s role:**

In [ ]:
prompt = f"""
You are OrderBot - an automated service to collect orders for a pizza restaurant.
You first greet the customer, then collects the order, and then asks if it's a pickup or delivery.
You wait to collect the entire order, then summarize it and check for a final time if the customer wants to add anything else.
"""

# Please note that we provided this prompt to system prompt argument
print(generate_response(prompt='Hi!', system_prompt=prompt))

## Prompt examples for Data Science / Machine Learning tasks:  

In [ ]:
prod_review = """\
Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \
super cute, and its face has a friendly look. It's \
a bit small for what I paid though. I think there \
might be other options that are bigger for the \
same price. It arrived a day earlier than expected, \
so I got to play with it myself before I gave it \
to her.\
"""

**1. Summarization with focus:**

In [ ]:
prompt = f"""
Generate a short summary of a product review from an ecommerce site \
to give feedback to the pricing deparmtment, responsible for determining the price of the product.
Summarize the review below in at most 30 words, and focusing on any aspects that are relevant to the price and perceived value.

Review:\n{prod_review}

Summary:
"""

print(generate_response(prompt=prompt))

**2. Information extraction:**

In [ ]:
prompt = f"""
Summarize the review below in order to give feedback to delivery department. Limit to 30 words.

Review:\n{prod_review}

Feedback to delivery department:\
"""

print(generate_response(prompt=prompt))

**3. Sentiment analysis:**

In [ ]:
prompt = f"""\
Classify sentiment of the following product review.
Give your answer as a single word, either "positive", "negative" or "neutral".

Review:{prod_review}
Sentiment:
"""

print(generate_response(prompt=prompt))

**4. Entity Extraction:**

In [ ]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

prompt = f"""
Identify the following entities from the text:

Company name
Date of contract
Sum of contract
Currency of a contract

The review is delimited with triple backticks.
Format your response as a JSON object with entities the keys and recoginized entities as values.
If the information isn't present, use "unknown" as the value. Make your response as short as possible.

Text: ```{lamp_review}```
"""

print(generate_response(prompt=prompt))

**5. Topic recognition (open topics):**

In [ ]:
story = """
In a recent survey conducted by the government,
public sector employees were asked to rate their level
of satisfaction with the department they work at.
The results revealed that NASA was the most popular
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings,
stating, "I'm not surprised that NASA came out on top.
It's a great place to work with amazing people and
incredible opportunities. I'm proud to be a part of
such an innovative organization."

The results were also welcomed by NASA's management team,
with Director Tom Johnson stating, "We are thrilled to
hear that our employees are satisfied with their work at NASA.
We have a talented and dedicated team who work tirelessly
to achieve our goals, and it's fantastic to see that their
hard work is paying off."

The survey also revealed that the
Social Security Administration had the lowest satisfaction
rating, with only 45% of employees indicating they were
satisfied with their job. The government has pledged to
address the concerns raised by employees in the survey and
work towards improving job satisfaction across all departments.
"""

In [ ]:
prompt = f"""
Determine five topics that are being discussed in the provided text.
Make each item one or two words long.
Give topics in Python list format.

Text sample: '''{story}'''
"""

print(generate_response(prompt=prompt))

**6. Topic recognition (closed list of topics):**

In [ ]:
topic_list = ["NASA", "Security", "Biology"]

prompt = f"""

Recognize a topic from the given topic list delimited in triple backticks
that is being discussed in the text, which is delimited by triple quotes. If you cannot determine between these 3, return 'other'.

Topic list: ```{topic_list}```

Text sample: '''{story}'''

Provide an answer only with one word, representing a determined topic.
"""

print(generate_response(prompt=prompt))

**7. Translation:**

In [ ]:
text = "Hello! How are you?"
system_prompt = "Translate provided text to Spanish"

print(generate_response(prompt=text, system_prompt=system_prompt))

**8. Tone transforming:**

In [ ]:
text = "Hi, man! Nice to see you. I will not be ready with my task, is it ok?"
prompt = f""" Translate the following from slang to a business letter: {text}. """

print(generate_response(prompt=prompt))

**9. Spellcheck / Grammar check:**

In [ ]:
text = "Hi, man! Nise to see you. I wont not be raedy with my task, it is ok?"

system_prompt = "Proofread and correct user message. Explain made corrections."

print(generate_response(prompt=text, system_prompt=system_prompt))

**10. Multiple tasks at the same time:**  
Be aware that a prompt with multiple tasks tends to be less stable and correct than several prompts each for its own task.

In [ ]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

prompt = f""" Identify the following items from the review text:

Sentiment (positive or negative)
Is the reviewer expressing anger? (true or false)
Item purchased by reviewer
Company that made the item

The review is delimited with triple backticks.
Format your response as a JSON object with "Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown"as the value.
Make your response as short as possible. Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
print(generate_response(prompt=prompt))

These prompts could be a good baseline for your DS/ML task. You can consider them and add your own modifications, just make sure to avoid hallucinations and make them stable.